In [16]:
DOCKER_REGISTRY = 'index.docker.io/dejangolubovic'
my_namespace = 'dejan-golubovic'

In [17]:
num_chief = 1 #number of Chief in TFJob 
num_ps = 1  #number of PS in TFJob 
num_workers = 2  #number of Worker in TFJob 
model_dir = "/mnt"
export_path = "/mnt/export" 
train_steps = "1000"
batch_size = "100"
learning_rate = "0.01"
pv_name = 'pvc-f8cc766b-aef7-4456-99bb-8c99621094af'
pvc_name = 'dejan-fairing-pvc-2'

In [18]:
import uuid
from kubeflow import fairing   
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource

tfjob_name = f'mnist-training-{uuid.uuid4().hex[:4]}'
print(tfjob_name)
output_map =  {
    "Dockerfile": "Dockerfile",
    "mnist.py": "mnist.py"
}

context_source = ''

command=["python",
         "/opt/mnist.py",
         "--tf-model-dir=" + model_dir,
         "--tf-export-dir=" + export_path,
         "--tf-train-steps=" + train_steps,
         "--tf-batch-size=" + batch_size,
         "--tf-learning-rate=" + learning_rate]

minio_context_source = MinioContextSource(
            endpoint_url='http://minio-service.kubeflow:9000/',
            minio_secret='minio',
            minio_secret_key='minio123',#
            region_name='region')

fairing.config.set_preprocessor('python', command=command, path_prefix="/app", output_map=output_map)
fairing.config.set_builder(name='cluster', registry=DOCKER_REGISTRY, \
                           context_source=minio_context_source, cleanup=True, \
                          pod_spec_mutators=[mounting_pvc(pvc_name=pvc_name, pvc_mount_path=model_dir)])
fairing.config.set_deployer(name='tfjob', namespace=my_namespace, stream_log=False, job_name=tfjob_name,
                            chief_count=num_chief, worker_count=num_workers, ps_count=num_ps, 
                            pod_spec_mutators=[mounting_pvc(pvc_name=pvc_name, pvc_mount_path=model_dir)])
fairing.config.run()
#fairing.config.run()

[W 201016 11:57:45 utils:51] The function mounting_pvc has been deprecated,                     please use `volume_mounts`
[W 201016 11:57:45 utils:51] The function mounting_pvc has been deprecated,                     please use `volume_mounts`
[I 201016 11:57:45 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7f6f0440a160>
[I 201016 11:57:45 config:136] Using builder: <kubeflow.fairing.builders.cluster.cluster.ClusterBuilder object at 0x7f6f0440a550>
[I 201016 11:57:45 config:138] Using deployer: <kubeflow.fairing.deployers.tfjob.tfjob.TfJob object at 0x7f6f04147ef0>
[I 201016 11:57:45 cluster:46] Building image using cluster builder.
[I 201016 11:57:45 base:107] Creating docker context: /tmp/fairing_context_6g4onakg
[W 201016 11:57:45 base:94] /tmp/fairing_dockerfile_n0wukmgj already exists in Fairing context, skipping...


mnist-training-a212


[W 201016 11:57:45 manager:298] Waiting for fairing-builder-qn754-d8rjt to start...
[W 201016 11:57:45 manager:298] Waiting for fairing-builder-qn754-d8rjt to start...
[W 201016 11:57:45 manager:298] Waiting for fairing-builder-qn754-d8rjt to start...
[I 201016 11:58:04 manager:304] Pod started running True


INFO[0001] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0003] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0004] Built cross stage deps: map[]
INFO[0004] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0005] Retrieving image manifest tensorflow/tensorflow:1.15.2-py3
INFO[0005] Executing 0 build triggers
INFO[0005] Unpacking rootfs as cmd ADD mnist.py /opt/mnist.py requires it.
INFO[0025] Taking snapshot of full filesystem...
INFO[0028] Resolving 27181 paths
INFO[0032] Using files from context: [/kaniko/buildcontext/mnist.py]
INFO[0032] ADD mnist.py /opt/mnist.py
INFO[0032] RUN chmod +x /opt/mnist.py
INFO[0032] cmd: /bin/sh
INFO[0032] args: [-c chmod +x /opt/mnist.py]
INFO[0032] Running: [/bin/sh -c chmod +x /opt/mnist.py]
INFO[0032] ENTRYPOINT ["/usr/bin/python"]
INFO[0032] CMD ["/opt/mnist.py"]
INFO[0032] Taking snapshot of full filesystem...
INFO[0032] Resolving 27182 paths


[W 201016 11:58:44 cluster:106] Cleaning up job fairing-builder-qn754...
[W 201016 11:58:44 job:101] The tfjob mnist-training-a212 launched.


(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7f6f0440a160>,
 <kubeflow.fairing.deployers.tfjob.tfjob.TfJob at 0x7f6f04147ef0>)

In [19]:
#tfjob_name = 'mnist-training-565f'
tfjob_namespace = 'dejan-golubovic'

In [20]:
from kubeflow.tfjob import TFJobClient
tfjob_client = TFJobClient()

tfjob_client.get(tfjob_name, namespace=tfjob_namespace)

{'apiVersion': 'kubeflow.org/v1',
 'kind': 'TFJob',
 'metadata': {'creationTimestamp': '2020-10-16T11:58:44Z',
  'generateName': 'fairing-tfjob-',
  'generation': 1,
  'labels': {'fairing-deployer': 'tfjob',
   'fairing-id': 'f11a7680-0fa6-11eb-8751-0a580a6404a8'},
  'name': 'mnist-training-a212',
  'namespace': 'dejan-golubovic',
  'resourceVersion': '30851855',
  'selfLink': '/apis/kubeflow.org/v1/namespaces/dejan-golubovic/tfjobs/mnist-training-a212',
  'uid': 'a25d015a-2eaf-48ed-990e-52de5da7abed'},
 'spec': {'tfReplicaSpecs': {'Chief': {'replicas': 1,
    'template': {'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
      'labels': {'fairing-deployer': 'tfjob',
       'fairing-id': 'f11a7680-0fa6-11eb-8751-0a580a6404a8'},
      'name': 'fairing-deployer'},
     'spec': {'containers': [{'command': ['python',
         '/opt/mnist.py',
         '--tf-model-dir=/mnt',
         '--tf-export-dir=/mnt/export',
         '--tf-train-steps=1000',
         '--tf-batch-size=1

In [21]:
tfjob_client.wait_for_job(tfjob_name, namespace=tfjob_namespace, watch=True)

mnist-training-a212            Created              2020-10-16T11:58:44Z          
mnist-training-a212            Created              2020-10-16T11:58:44Z          
mnist-training-a212            Running              2020-10-16T11:59:15Z          
mnist-training-a212            Running              2020-10-16T11:59:15Z          
mnist-training-a212            Running              2020-10-16T11:59:15Z          
mnist-training-a212            Succeeded            2020-10-16T11:59:19Z          


In [22]:
tfjob_client.is_job_succeeded(tfjob_name, namespace=tfjob_namespace)

True

In [ ]:
#tfjob_client.get_logs(tfjob_name, namespace=tfjob_namespace)

In [23]:
from kubeflow.fairing.deployers.kfserving.kfserving import KFServing
isvc_name = f'dejan-mnist-service-{uuid.uuid4().hex[:4]}'
print(isvc_name)
isvc = KFServing('tensorflow', namespace=tfjob_namespace, isvc_name=isvc_name,
                 default_storage_uri='pvc://' + pvc_name + '/export')
isvc.deploy(isvc.generate_isvc())

dejan-mnist-service-1efc
NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
dejan-mnist-servi... Unknown                                                                                      
dejan-mnist-servi... False                                                                                        
dejan-mnist-servi... False                                                                                        
dejan-mnist-servi... False                                                                                        
dejan-mnist-servi... True       100                             http://dejan-mnist-service-1efc.dejan-golubovic...


[I 201016 12:00:14 kfserving:127] Deployed the InferenceService dejan-mnist-service-1efc successfully.


'dejan-mnist-service-1efc'

In [24]:
from kfserving import KFServingClient
kfserving_client = KFServingClient()
kfserving_client.get(namespace=tfjob_namespace)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'items': [{'apiVersion': 'serving.kubeflow.org/v1alpha2',
   'kind': 'InferenceService',
   'metadata': {'creationTimestamp': '2020-10-16T11:42:54Z',
    'generateName': 'fairing-kfserving-',
    'generation': 1,
    'name': 'dejan-mnist-service',
    'namespace': 'dejan-golubovic',
    'resourceVersion': '30845834',
    'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/dejan-golubovic/inferenceservices/dejan-mnist-service',
    'uid': '334148ad-c3bf-4d07-bda3-4b0407483b69'},
   'spec': {'default': {'predictor': {'tensorflow': {'resources': {'limits': {'cpu': '1',
         'memory': '2Gi'},
        'requests': {'cpu': '1', 'memory': '2Gi'}},
       'runtimeVersion': '1.14.0',
       'storageUri': 'pvc://dejan-fairing-pvc-2/export'}}}},
   'status': {'address': {'url': 'http://dejan-mnist-service.dejan-golubovic.svc.cluster.local/v1/models/dejan-mnist-service:predict'},
    'canary': {},
    'conditions': [{'lastTransitionTime':

In [25]:
mnist_isvc = kfserving_client.get(isvc_name, namespace=tfjob_namespace)
mnist_isvc_name = mnist_isvc['metadata']['name']
mnist_isvc_endpoint = mnist_isvc['status'].get('url', '')
print("MNIST Service Endpoint: " + mnist_isvc_endpoint)

MNIST Service Endpoint: http://dejan-mnist-service-1efc.dejan-golubovic.example.com/v1/models/dejan-mnist-service-1efc


In [ ]:
#ISTIO_CLUSTER_IP=!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.clusterIP}'
#CLUSTER_IP=ISTIO_CLUSTER_IP[0]
CLUSTER_IP = '10.254.73.13'
MODEL_HOST=f"Host: {mnist_isvc_name}.{tfjob_namespace}.example.com"
print(MODEL_HOST)
print('curl -v -H'+MODEL_HOST+' http://'+CLUSTER_IP+'/v1/models/'+mnist_isvc_name+':predict -d @./input.json')
!curl -v -H "{MODEL_HOST}" http://{CLUSTER_IP}/v1/models/{mnist_isvc_name}:predict -d @./input.json

In [ ]:
tfjob_client.delete(tfjob_name, namespace=my_namespace)